# MUSI - Image Analysis  

## Project 2: Step-by-step implementation of a tracking algorithm   

### **Members:** Umar Faruk Abdullahi, Raixa A. Madueño
---

### Establish the necessary libraries and constants

In [1]:
#Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
#Constant
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

## Submethods 
---
### Obtain contours: External contours

In [2]:
def obtain_contours(foreground_frame):
    contours, _ = cv2.findContours(foreground_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

### Manage contour boxes

In [3]:
def boxes_overlap(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    return not (x1 + w1 < x2 or x1 > x2 + w2 or y1 + h1 < y2 or y1 > y2 + h2)

def not_exceed_max_area(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    w= max(x1 + w1, x2 + w2) - x
    h = max(y1 + h1, y2 + h2) - y
    total_area=w*h
    if total_area < 18000:
        return True
    else:
        return False
def merge_boxes(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x = min(x1, x2)
    y = min(y1, y2)
    w = max(x1 + w1, x2 + w2) - x
    h = max(y1 + h1, y2 + h2) - y
    return (x, y, w, h)

### Apply preprocess to the video 
---
Video initialization

In [4]:
video_path="FroggerHighway.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

In [5]:
#Reset video to the first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)
    # Kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))
    # Apply dilation
    fgmask = cv2.dilate(fgmask, kernel, iterations=3)
    # Apply erosion
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    
    contours=obtain_contours(fgmask)
    filtered_contours=[]
    boxes = []
    for contour in contours:
        if 1000 < cv2.contourArea(contour):
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)
            
            if aspect_ratio > 1.5:
                boxes.append((x, y-5, w, h))

    # Merge and adjust boxes
    for box in boxes:
        overlaps = False
        for i, prev_box in enumerate(filtered_contours):
            if boxes_overlap(box, prev_box) & not_exceed_max_area(box, prev_box):
                overlaps = True
                # Merge boxes if they overlap significantly
                merged_box = merge_boxes(box, prev_box)
                filtered_contours[i] = merged_box
                break
        if not overlaps:
            filtered_contours.append(box)

    for box in filtered_contours:
        x, y, w, h = box
        area=w*h
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)  
        #cv2.putText(fgmask, f'Area: {area}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
